In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import folium
from rasterio.plot import reshape_as_image
import branca.colormap as cm
from rasterio.transform import from_origin
from shapely.geometry import Point


In [2]:
# Earthquake CSV (must have lat, lon, magnitude)
eq = pd.read_excel(r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\cearthquakes.xlsx")  # columns: lat, lon, mag
gdf = gpd.GeoDataFrame(eq, geometry=gpd.points_from_xy(eq.Longitude, eq.Latitude), crs="EPSG:4326")
gdf = gdf.to_crs(epsg=32645)  # UTM for Nepal (meters)


In [3]:
gdf.head()

,Latitude,Longitude,Magnitude,Epicenter,AD_Date,Local_Time,geometry
0,27.54,87.14,4.4,Sankhuwasabha,2025-08-22,23:15,POINT (513823.171 3046255.322)
1,27.70,86.53,4.0,Ramechap,2025-08-17,15:43,POINT (453660.769 3064058.907)
2,28.96,82.12,5.5,Jajarkot,2025-04-04,20:10,POINT (24197.907 3213380.269)
3,28.95,82.12,5.2,Jajarkot,2025-04-04,20:07,POINT (24152.023 3212270.212)
4,30.02,80.84,4.0,Darchula,2025-04-03,17:04,POINT (-94574.032 3337031.203)


In [13]:
# Define bounds and resolution
xmin, ymin, xmax, ymax = gdf.total_bounds
pixel_size = 1000# meters
n_cols = int((xmax - xmin) / pixel_size)
n_rows = int((ymax - ymin) / pixel_size)

# Create grid coordinates
x = np.linspace(xmin + pixel_size/2, xmax - pixel_size/2, n_cols)
y = np.linspace(ymax - pixel_size/2, ymin + pixel_size/2, n_rows)  # top to bottom
xx, yy = np.meshgrid(x, y)
hazard_grid = np.zeros_like(xx)


In [14]:
#smaller the decay value, the wider the spread of intensity and vice versa
decay = 2

for idx, row in gdf.iterrows():
    #  calculates Epicenter coordinates
    ex, ey = row.geometry.x, row.geometry.y
    mag = row['Magnitude']
    
    # Distance from epicenter to all grid cells
    dist = np.sqrt((xx - ex)**2 + (yy - ey)**2)
    
    # Add intensity contribution
    #Converts distance and magnitude into a hazard value
    hazard_grid += mag / (dist/1000 + 1)**decay  # dist in km

In [15]:
#Normalization of hazard values between 0 and 1
hazard_grid_norm = (hazard_grid - hazard_grid.min()) / (hazard_grid.max() - hazard_grid.min())


In [16]:
# Open population raster (WorldPop)
pop_raster = r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\npl_pop_2025_CN_100m_R2025A_v1.tif"
with rasterio.open(pop_raster) as pop_src: #safeway to open and close file
    pop_data = pop_src.read(1) #reads actual pixel value from file, reads only 1st layer
    pop_data = np.where(pop_data < 0, 0, pop_data)
    pop_transform = pop_src.transform      #for georeferencing
    pop_crs = pop_src.crs

    # Simple resample to match hazard grid size (use nearest for simplicity)
    %pip install -q scikit-image
    from skimage.transform import resize
    pop_resampled = resize(pop_data, hazard_grid_norm.shape, order=0, preserve_range=True) 
    #Takes population data and resizes it to match the hazard grid shape
    




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


After above code runs I have two grids hazard_grid_norm and pop_resampled that are perfectly aligned.


In [17]:
exposure = hazard_grid_norm * pop_resampled
total_exposed = exposure.sum()
print(f"Estimated exposed population: {int(total_exposed)}")


Estimated exposed population: 29589


In [18]:
# Exposure by hazard class (Low / Medium / High)
low = exposure[exposure < 0.33].sum()
medium = exposure[(exposure >= 0.33) & (exposure < 0.66)].sum()
high = exposure[exposure >= 0.66].sum()

print("Exposure by hazard class:")
print(f"Low: {int(low)}, Medium: {int(medium)}, High: {int(high)}")


Exposure by hazard class:
Low: 10207, Medium: 3708, High: 15674


In [19]:
transform = from_origin(xmin, ymax, pixel_size, pixel_size)
with rasterio.open(
    "hazard_intensity.tif", "w",           #open and writes tif file
    driver="GTiff",
    height=hazard_grid_norm.shape[0],
    width=hazard_grid_norm.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:32645',
    transform=transform
) as dst:
    dst.write(hazard_grid_norm.astype('float32'), 1)

with rasterio.open(
    "exposure.tif", "w",
    driver="GTiff",
    height=exposure.shape[0],
    width=exposure.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:32645',
    transform=transform
) as dst:
    dst.write(exposure.astype('float32'), 1)


In [20]:
from rasterstats import zonal_stats
import geopandas as gpd

# Load district shapefile
districts = gpd.read_file(r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\Nepal Districts Shapefile Download\03_DISTRICT\DISTRICT.shp")
districts = districts.to_crs("EPSG:32645")  # Match raster CRS

# Hazard stats (per district)
haz_stats = zonal_stats(districts, "hazard_intensity.tif", stats=["mean", "max"], geojson_out=True)

# Exposure stats (per district)
exp_stats = zonal_stats(districts, "exposure.tif", stats=["sum"], geojson_out=True)

# Attach stats to GeoDataFrame
districts["hazard_mean"] = [f["properties"]["mean"] for f in haz_stats]
districts["hazard_max"] = [f["properties"]["max"] for f in haz_stats]
districts["exposed_pop"] = [f["properties"]["sum"] for f in exp_stats]

# Simple Risk Index
districts["risk_index"] = districts["hazard_mean"] * districts["exposed_pop"]


In [ ]:
# Ensure 'districts' GeoDataFrame is loaded and CRS is set
if 'districts' not in locals():
    districts = gpd.read_file(r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\Nepal Districts Shapefile Download\03_DISTRICT\DISTRICT.shp")
    districts = districts.to_crs("EPSG:32645")

from rasterstats import zonal_stats

# Calculate total population in each district from the raster
pop_stats = zonal_stats(
    districts, 
    r"D:\Projects\GIS\earthquake-risk-mapping-in-Nepal\data\npl_pop_2025_CN_100m_R2025A_v1.tif", 
    stats=["sum"], 
    geojson_out=True
)

# Add the population sum to your districts GeoDataFrame
districts["population"] = [f["properties"]["sum"] for f in pop_stats]

MemoryError: Unable to allocate 19.3 PiB for an array with shape (1, 68640031, 79341799) and data type float32

In [ ]:
districts.head()

,OBJECTID,STATE_CODE,DISTRICT,GaPa_NaPa,Type_GN,Province,Area,Shape_Leng,Shape_Area,geometry,hazard_mean,hazard_max,exposed_pop,risk_index,hazard_norm,risk_norm,population
0,1,7,DOTI,Adharsha,Gaunpalika,Sudur Pashchim,0.0,282908.088904,2.054599e+09,"POLYGON ((81.03282 29.18620, 81.03295 29.18651...",0.025581,0.312059,0.000000,0.000000,0.079843,0.000000,2.083943e+05
1,2,3,RAMECHHAP,Doramba,Gaunpalika,Bagmati,0.0,295289.035440,1.565522e+09,"POLYGON ((86.22432 27.38306, 86.22421 27.38301...",0.113920,0.369223,754.381226,85.938900,0.355564,0.074361,1.701506e+05
2,3,5,DANG,Babai,Gaunpalika,5,0.0,352004.991843,3.059781e+09,"POLYGON ((82.74840 27.82304, 82.74881 27.82289...",0.007402,0.042257,0.000000,0.000000,0.023104,0.000000,6.850648e+05
3,4,5,RUPANDEHI,Butwal,Upamahanagarpalika,5,0.0,229898.404296,1.304475e+09,"POLYGON ((83.40117 27.42033, 83.40096 27.42020...",0.006734,0.008266,0.000000,0.000000,0.021019,0.000000,1.132744e+06
4,5,3,SINDHULI,Dudhouli,Nagarpalika,Bagmati,0.0,352777.949556,2.486021e+09,"POLYGON ((86.30632 27.10883, 86.30639 27.10839...",0.032530,0.067117,621.306763,20.210952,0.101531,0.017488,3.023759e+05


In [ ]:
# Exposed population = population * normalized hazard (0-1)
districts["hazard_norm"] = districts["hazard_mean"] / districts["hazard_mean"].max()

# Total exposed population
total_exposed = districts["exposed_pop"].sum()
print(f"Estimated exposed population: {int(total_exposed)}")


Estimated exposed population: 19729


In [ ]:
districts["risk_index"] = districts["hazard_mean"] * districts["exposed_pop"]


In [ ]:
districts.head()

,OBJECTID,STATE_CODE,DISTRICT,GaPa_NaPa,Type_GN,Province,Area,Shape_Leng,Shape_Area,geometry,hazard_mean,hazard_max,exposed_pop,risk_index,hazard_norm
0,1,7,DOTI,Adharsha,Gaunpalika,Sudur Pashchim,0.0,282908.088904,2.054599e+09,"POLYGON ((-80712.117 3243395.863, -80697.384 3...",0.025581,0.312059,0.000000,0.000000,0.079843
1,2,3,RAMECHHAP,Doramba,Gaunpalika,Bagmati,0.0,295289.035440,1.565522e+09,"POLYGON ((423301.723 3029102.859, 423290.918 3...",0.113920,0.369223,754.381226,85.938900,0.355564
2,3,5,DANG,Babai,Gaunpalika,5,0.0,352004.991843,3.059781e+09,"POLYGON ((81072.662 3084862.831, 81112.370 308...",0.007402,0.042257,0.000000,0.000000,0.023104
3,4,5,RUPANDEHI,Butwal,Upamahanagarpalika,5,0.0,229898.404296,1.304475e+09,"POLYGON ((144143.103 3038143.157, 144121.643 3...",0.006734,0.008266,0.000000,0.000000,0.021019
4,5,3,SINDHULI,Dudhouli,Nagarpalika,Bagmati,0.0,352777.949556,2.486021e+09,"POLYGON ((431241.656 2998678.491, 431248.463 2...",0.032530,0.067117,621.306763,20.210952,0.101531


In [ ]:
# Normalize risk index for visualization (0-1)
districts["risk_norm"] = (districts["risk_index"] - districts["risk_index"].min()) / (districts["risk_index"].max() - districts["risk_index"].min())

In [ ]:
districts

,OBJECTID,STATE_CODE,DISTRICT,GaPa_NaPa,Type_GN,Province,Area,Shape_Leng,Shape_Area,geometry,hazard_mean,hazard_max,exposed_pop,risk_index,hazard_norm
0,1,7,DOTI,Adharsha,Gaunpalika,Sudur Pashchim,0.0,282908.088904,2.054599e+09,"POLYGON ((-80712.11684 3243395.86322, -80697.3...",0.025581,0.312059,0.000000,0.000000,0.079843
1,2,3,RAMECHHAP,Doramba,Gaunpalika,Bagmati,0.0,295289.035440,1.565522e+09,"POLYGON ((423301.72283 3029102.85891, 423290.9...",0.113920,0.369223,754.381226,85.938900,0.355564
2,3,5,DANG,Babai,Gaunpalika,5,0.0,352004.991843,3.059781e+09,"POLYGON ((81072.66193 3084862.83060, 81112.370...",0.007402,0.042257,0.000000,0.000000,0.023104
3,4,5,RUPANDEHI,Butwal,Upamahanagarpalika,5,0.0,229898.404296,1.304475e+09,"POLYGON ((144143.10273 3038143.15668, 144121.6...",0.006734,0.008266,0.000000,0.000000,0.021019
4,5,3,SINDHULI,Dudhouli,Nagarpalika,Bagmati,0.0,352777.949556,2.486021e+09,"POLYGON ((431241.65649 2998678.49092, 431248.4...",0.032530,0.067117,621.306763,20.210952,0.101531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,3,DHADING,Benighat Rorang,Gaunpalika,Bagmati,0.0,359568.911316,1.906729e+09,"POLYGON ((325560.93655 3073299.19634, 325586.3...",0.134179,0.632640,884.858093,118.729184,0.418796
73,74,1,TAPLEJUNG,Aathrai Tribeni,Gaunpalika,1,0.0,301482.064043,3.643851e+09,"POLYGON ((605532.63089 3035538.47878, 605514.6...",0.052154,0.308263,907.598755,47.334893,0.162782
74,75,5,BANKE,Baijanath,Gaunpalika,5,0.0,235805.166929,1.882279e+09,"POLYGON ((3906.96850 3098920.51756, 3894.17350...",0.008830,0.222315,0.000000,0.000000,0.027559
75,76,5,PALPA,Bagnaskali,Gaunpalika,5,0.0,270596.477602,1.461895e+09,"POLYGON ((180758.73846 3090695.56242, 180805.5...",0.009670,0.013737,5.814373,0.056226,0.030182


In [ ]:
# Ensure the districts GeoDataFrame has a CRS first (UTM 32645)
districts = districts.set_crs(epsg=32645, allow_override=True)

# Convert to latitude/longitude
districts = districts.to_crs(epsg=4326)

# Now the geometries are in lat/lon
print(districts.geometry.head())
print(districts.geometry.total_bounds)  # should be roughly [80,26,88,30] for Nepal


0    POLYGON ((81.03282 29.18620, 81.03295 29.18651...
1    POLYGON ((86.22432 27.38306, 86.22421 27.38301...
2    POLYGON ((82.74840 27.82304, 82.74881 27.82289...
3    POLYGON ((83.40117 27.42033, 83.40096 27.42020...
4    POLYGON ((86.30632 27.10883, 86.30639 27.10839...
Name: geometry, dtype: geometry
[80.05847201 26.3477645  88.20155484 30.47296883]


In [ ]:
districts

,OBJECTID,STATE_CODE,DISTRICT,GaPa_NaPa,Type_GN,Province,Area,Shape_Leng,Shape_Area,geometry,hazard_mean,hazard_max,exposed_pop,risk_index,hazard_norm
0,1,7,DOTI,Adharsha,Gaunpalika,Sudur Pashchim,0.0,282908.088904,2.054599e+09,"POLYGON ((81.03282 29.18620, 81.03295 29.18651...",0.025581,0.312059,0.000000,0.000000,0.079843
1,2,3,RAMECHHAP,Doramba,Gaunpalika,Bagmati,0.0,295289.035440,1.565522e+09,"POLYGON ((86.22432 27.38306, 86.22421 27.38301...",0.113920,0.369223,754.381226,85.938900,0.355564
2,3,5,DANG,Babai,Gaunpalika,5,0.0,352004.991843,3.059781e+09,"POLYGON ((82.74840 27.82304, 82.74881 27.82289...",0.007402,0.042257,0.000000,0.000000,0.023104
3,4,5,RUPANDEHI,Butwal,Upamahanagarpalika,5,0.0,229898.404296,1.304475e+09,"POLYGON ((83.40117 27.42033, 83.40096 27.42020...",0.006734,0.008266,0.000000,0.000000,0.021019
4,5,3,SINDHULI,Dudhouli,Nagarpalika,Bagmati,0.0,352777.949556,2.486021e+09,"POLYGON ((86.30632 27.10883, 86.30639 27.10839...",0.032530,0.067117,621.306763,20.210952,0.101531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,3,DHADING,Benighat Rorang,Gaunpalika,Bagmati,0.0,359568.911316,1.906729e+09,"POLYGON ((85.22971 27.77288, 85.22997 27.77277...",0.134179,0.632640,884.858093,118.729184,0.418796
73,74,1,TAPLEJUNG,Aathrai Tribeni,Gaunpalika,1,0.0,301482.064043,3.643851e+09,"POLYGON ((88.06782 27.43923, 88.06763 27.43902...",0.052154,0.308263,907.598755,47.334893,0.162782
74,75,5,BANKE,Baijanath,Gaunpalika,5,0.0,235805.166929,1.882279e+09,"POLYGON ((81.96168 27.92315, 81.96156 27.92280...",0.008830,0.222315,0.000000,0.000000,0.027559
75,76,5,PALPA,Bagnaskali,Gaunpalika,5,0.0,270596.477602,1.461895e+09,"POLYGON ((83.75700 27.90307, 83.75747 27.90320...",0.009670,0.013737,5.814373,0.056226,0.030182


In [ ]:
import geopandas as gpd
import folium
import branca.colormap as cm


nepal_bounds = districts.total_bounds
print(nepal_bounds)

# 4️⃣ Remove empty or null geometries
districts = districts[districts.geometry.notnull() & ~districts.geometry.is_empty]

# 5️⃣ Make sure risk_norm exists and is 0-1
districts["risk_norm"] = (districts["risk_index"] - districts["risk_index"].min()) / \
                         (districts["risk_index"].max() - districts["risk_index"].min())
districts["risk_norm"] = districts["risk_norm"].fillna(0).clip(0,1)


# 7️⃣ Create Folium map
m = folium.Map(location=[28.2, 84.0], zoom_start=7, tiles="CartoDB positron")
m.fit_bounds([[nepal_bounds[1], nepal_bounds[0]], [nepal_bounds[3], nepal_bounds[2]]])

colors= ["#f0f0f0", "#ffffb2", "#fecc5c", "#fd8d3c", "#e31a1c", "#800026"]
colormap = cm.LinearColormap(colors=colors, vmin=0, vmax=1, caption="Earthquake Risk Index")

folium.GeoJson(
    districts,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["risk_norm"]),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["DISTRICT", "population", "hazard_mean", "exposed_pop", "risk_index"],
        aliases=["District:", "Population:", "Mean Hazard:", "Exposed Pop:", "Risk Index:"],
        localize=True,
        sticky=True,
        labels=True
    )
).add_to(m)

colormap.add_to(m)
m.save("earthquake_risk_map.html")
print("✅ Map saved. Nepal districts should now appear.")




✅ Map saved. Nepal districts should now appear.
